# Tutorial

In [1]:
import pandas as pd
from glob import glob
from os.path import splitext
from os.path import basename

In [100]:
editions_metadata = pd.read_csv('./editions list.csv', encoding='utf-8', index_col=0)

I thought it might be appropriate to add the text of the newspaper, which we cleaned, to the DataFrame for a more complete view.

In [83]:
def get_text_number(filepath):
    return int(splitext(basename(filepath))[0])

def read_file(filename):
    with open(filename, encoding='utf8') as infile:
        contents = infile.read()
    return contents

corpus = []
filenames = sorted(glob('./clean_texts/*.txt'), key=get_text_number)
for filename in filenames:
    text = read_file(filename)
    corpus.append(text)

In [101]:
editionlist_texts = editions_metadata.assign(text=corpus)
editionlist_texts.head()
#editionlist_texts.to_csv('editionlist_texts.csv')

,title,ideology,headquarter,audience,region,format,year,month,day,link,text
id,,,,,,,,,,,
1,Ahora,socialist,madrid,adult,national,daily,1931,4,9,https://hemerotecadigital.bne.es/hd/es/viewer?...,. autómata que hace un par de años al público ...
2,Ahora,socialist,madrid,adult,national,daily,1932,4,15,https://hemerotecadigital.bne.es/hd/es/viewer?...,nota de gran jornada republicana del ha la rev...
3,Ahora,socialist,madrid,adult,national,daily,1932,9,16,https://hemerotecadigital.bne.es/hd/es/viewer?...,del la banda e de la de la al y republicano do...
4,Ahora,socialist,madrid,adult,national,daily,1935,1,9,https://hemerotecadigital.bne.es/hd/es/viewer?...,consecuencia los que últimamente algunas zonas...
5,Ahora,socialist,madrid,adult,national,daily,1935,11,15,https://hemerotecadigital.bne.es/hd/es/viewer?...,concentraciones de tropas en y movimiento de a...


#### Below is the way we worked. But be careful, it may not work for your texts. Take inspiration but write your own code! 

In [ ]:
import nltk

nltk.download("punkt")
nltk.download("cess_esp")

def retokenize(tokenized_text):
    tokenized = []
    is_broken_word = False
    temp_word = ""
    for token in tokenized_text:
        if len(token) == 1:
            if not is_broken_word:
                is_broken_word = True
                temp_word = token
            else:
                temp_word += token
        else:
            if is_broken_word:
                tokenized.append(temp_word)
                temp_word = ""
                is_broken_word = False
            tokenized.append(token)
    return tokenized

spanish_corpus = {word.lower() for word in nltk.corpus.cess_esp.words()}

for filename in glob('texts/*.txt'):
    text = read_file(filename)
    tokenized_text = nltk.tokenize.word_tokenize(text)
    tokenized_text = retokenize(tokenized_text)
    clean_texts = [word for word in tokenized_text if word in spanish_corpus]
    with open(f'clean_{filename}', 'w', encoding = 'utf-8') as fp:
        for item in clean_texts:
            fp.write(f'{item} ')

Stylo and Gephi

In [ ]:
editions_list = []
for edition in glob('./clean_texts/*.txt'):
    editions_list.append(edition)
    
editions_metadata = pd.read_csv('./editions list.csv', encoding='utf-8', index_col=0)
editions_metadata

def create_subcorpus(feature):
    if not os.path.exists("./stylo/"):
        os.makedirs("./stylo/")
    subcorpus = str("./stylo/" + feature + "/corpus")
    if not os.path.exists(subcorpus):
        os.makedirs(subcorpus)
    for edition in editions_list:
        edition_id = int(os.path.splitext(basename(edition))[0])
        feature_lookup = editions_metadata.iloc[edition_id-1][feature]
        edition_id = str(edition_id)
        with open(edition, "r") as oldfile, open(subcorpus + "/" + feature_lookup + "_" + edition_id + ".txt", "w") as newfile:
            for line in oldfile:
                newfile.write(line)